In [1]:
import cv2
import numpy as np
import math

def show_image(image, title='Image'):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 读取图像
image = cv2.imread('./wlsj/VSLOT2(1).jpg')

# 检查图像是否成功加载
if image is None:
    print("图像加载失败，请检查路径。")
else:
    # 转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    show_image(gray, 'Gray Image')

    # 增强对比度，进行对比度拉伸
    gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)
    # 各个参数的含义：cv2.convertScaleAbs(gray, alpha=1.5, beta=0)表示对图像进行对比度拉伸，alpha表示对比度因子，beta表示亮度因子。
    # alpha越大，对比度越大；beta越大，亮度越大。
    show_image(gray, 'Contrast Enhanced Image')

    # 应用高斯模糊，减少噪声
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    show_image(blurred, 'Blurred Image')

    # 使用Canny边缘检测
    edges = cv2.Canny(blurred, 30, 100)
    show_image(edges, 'edges')

    # 找到边缘的轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 找到最大的轮廓，假设它是零件的轮廓
    max_contour = max(contours, key=cv2.contourArea)

    # 创建掩模
    mask = np.zeros_like(gray)  # 使用与灰度图相同大小和类型的掩模
    # 掩膜的作用是只保留我们感兴趣的区域，即零件的轮廓

    # 在掩模上绘制零件轮廓
    cv2.drawContours(mask, [max_contour], -1, (255), thickness=cv2.FILLED)
    
    # 自适应二值化处理
    binary = cv2.adaptiveThreshold(mask, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 6)
    # 各个参数的含义：cv2.ADAPTIVE_THRESH_GAUSSIAN_C表示使用高斯窗口，cv2.THRESH_BINARY_INV表示反二值化，11表示窗口大小，2表示C值，c用于调整阈
    # 窗口大小越大，二值化效果越好，但可能会丢失细节；C值越大，二值化效果越差，但可能会保留更多细节。
    show_image(binary, 'Binary Image')


    
    # 形态学操作：开运算去除噪声
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # 再次使用膨胀操作使槽点区域更明显
    morph = cv2.dilate(morph, kernel, iterations=2)
    
    # 查找轮廓
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 存储槽点坐标
    slot_points = []
    
    # 遍历所有轮廓
    for contour in contours:
        # 过滤轮廓，根据面积和形状筛选出槽点
        area = cv2.contourArea(contour)
        if area > 10:  # 根据实际情况调整面积阈值
            # 获取轮廓的最小外接矩形
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # 计算槽点的中心位置
            cx = int(rect[0][0])
            cy = int(rect[0][1])
            slot_points.append((cx, cy))
            
            # 绘制轮廓和中心点
            # cv2.drawContours(image, [box], 0, (0, 255, 0), 2)
            cv2.circle(image, (cx, cy), 5, (0,255,0), -1)
    
    # 按X坐标对槽点排序（假设槽点从左到右排列）
    slot_points = sorted(slot_points, key=lambda x: x[0])
    
    # 计算槽点之间的距离、角度和深度
    if len(slot_points) > 1:
        for i in range(1, len(slot_points)):
            prev_point = slot_points[i - 1]
            current_point = slot_points[i]
            
            # 计算槽点间的距离
            distance = math.sqrt((current_point[0] - prev_point[0])**2 + (current_point[1] - prev_point[1])**2)
            
            # 计算槽点间的角度
            angle = math.degrees(math.atan2(current_point[1] - prev_point[1], current_point[0] - prev_point[0]))
            
            # 假设槽深度为槽点到零件底部的距离（根据实际情况调整）
            depth = abs(current_point[1] - max([point[1] for point in slot_points]))
            
            # print(f"槽点 {i}: 距离 = {distance:.2f}，角度 = {angle:.2f}°，深度 = {depth:.2f}")
            
            # 在图像上显示槽点的连接线和标注
            # cv2.line(image, prev_point, current_point, (255, 0, 0), 1)
            # cv2.putText(image, f"D:{distance:.2f}", (prev_point[0], prev_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            # cv2.putText(image, f"A:{angle:.2f}", (prev_point[0], prev_point[1] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            # cv2.putText(image, f"H:{depth:.2f}", (prev_point[0], prev_point[1] - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    # 显示结果
    show_image(image, 'Detected Slots with Distances, Angles, and Depths')
    
    # # 打印槽点坐标
    for i, point in enumerate(slot_points):
        print(f"槽点 {i+1} 坐标: {point}")


C:\Users\SU_hole\AppData\Local\Temp\ipykernel_77952\1965277921.py:77: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


槽点 1 坐标: (65, 296)
槽点 2 坐标: (125, 327)
槽点 3 坐标: (176, 294)
槽点 4 坐标: (228, 325)
槽点 5 坐标: (245, 302)
槽点 6 坐标: (279, 292)
槽点 7 坐标: (329, 323)
槽点 8 坐标: (380, 290)
槽点 9 坐标: (431, 322)
槽点 10 坐标: (482, 289)
槽点 11 坐标: (532, 320)
槽点 12 坐标: (533, 393)
槽点 13 坐标: (583, 287)
槽点 14 坐标: (634, 318)
槽点 15 坐标: (685, 285)
槽点 16 坐标: (735, 317)
槽点 17 坐标: (785, 283)
槽点 18 坐标: (837, 315)
槽点 19 坐标: (888, 281)
槽点 20 坐标: (938, 313)


In [15]:
import cv2
import numpy as np
import math

def show_image(image, title='Image'):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def point_to_line_distance(point, line_start, line_end):
    """
    计算点到线段的最短距离
    :param point: (x, y)
    :param line_start: (x1, y1)
    :param line_end: (x2, y2)
    :return: 距离
    """
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end
    num = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    den = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return num / den

# 读取图像
image = cv2.imread('./wlsj/VSLOT2(1).jpg')

# 检查图像是否成功加载
if image is None:
    print("图像加载失败，请检查路径。")
else:
    # 转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 增强对比度，进行对比度拉伸
    gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

    # 应用高斯模糊，减少噪声
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 使用Canny边缘检测
    edges = cv2.Canny(blurred, 30, 100)

    # 找到边缘的轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 找到最大的轮廓，假设它是零件的轮廓
    max_contour = max(contours, key=cv2.contourArea)

    # 创建掩模
    mask = np.zeros_like(gray)

    # 在掩模上绘制零件轮廓
    cv2.drawContours(mask, [max_contour], -1, (255), thickness=cv2.FILLED)
    
    # 自适应二值化处理
    binary = cv2.adaptiveThreshold(mask, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 6)
    
    # 形态学操作：开运算去除噪声
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # 再次使用膨胀操作使槽点区域更明显
    morph = cv2.dilate(morph, kernel, iterations=2)
    
    # 查找轮廓
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 存储槽点坐标
    slot_points = []
    
    # 遍历所有轮廓
    for contour in contours:
        # 过滤轮廓，根据面积和形状筛选出槽点
        area = cv2.contourArea(contour)
        if area > 10:  # 根据实际情况调整面积阈值
            # 获取轮廓的最小外接矩形
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # 计算槽点的中心位置
            cx = int(rect[0][0])
            cy = int(rect[0][1])
            slot_points.append((cx, cy))
    
    # 按X坐标对槽点排序（假设槽点从左到右排列）
    slot_points = sorted(slot_points, key=lambda x: x[0])
    
    # 删除编号为5和12的槽点（注意：编号从1开始）
    points_to_remove = [5, 12]
    slot_points = [point for i, point in enumerate(slot_points, start=1) if i not in points_to_remove]
    
    # 标号槽点
    for i, point in enumerate(slot_points):
        # 绘制中心点
        cv2.circle(image, point, 5, (0, 255, 0), -1)
        # 绘制标号
        cv2.putText(image, f"{i+1}", (point[0], point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # 偶数点与奇数点的索引分组
    even_points = [point for i, point in enumerate(slot_points, start=1) if i % 2 == 0]
    odd_points = [point for i, point in enumerate(slot_points, start=1) if i % 2 != 0]
    
    # 连线所有偶数点并计算距离
    for i in range(len(even_points) - 1):
        start_point = even_points[i]
        end_point = even_points[i + 1]
        # 计算距离
        distance = math.sqrt((end_point[0] - start_point[0])**2 + (end_point[1] - start_point[1])**2)
        # 标注距离，保留两位小数
        mid_x = (start_point[0] + end_point[0]) // 2
        mid_y = (start_point[1] + end_point[1]) // 2
        cv2.putText(image, f"{distance:.2f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        # 绘制连线
        cv2.line(image, start_point, end_point, (255, 0, 0), 1)
    
    # 连线所有奇数点
    for i in range(len(odd_points) - 1):
        start_point = odd_points[i]
        end_point = odd_points[i + 1]
        # 绘制连线
        cv2.line(image, start_point, end_point, (0, 0, 255), 1)
    
    # 计算偶数点到奇数点连成的线的距离
    for even_point in even_points:
        min_distance = float('inf')
        for i in range(len(odd_points) - 1):
            line_start = odd_points[i]
            line_end = odd_points[i + 1]
            distance = point_to_line_distance(even_point, line_start, line_end)
            min_distance = min(min_distance, distance)
        # 在偶数点旁边标注最近距离
        cv2.putText(image, f"{min_distance:.2f}", (even_point[0] + 10, even_point[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    # 显示结果
    show_image(image, 'Even and Odd Points Connection')
    
    # 打印偶数点与奇数点信息
    print("偶数点坐标:")
    for i, point in enumerate(even_points, start=1):
        print(f"偶数点 {i}: {point}")
    print("\n奇数点坐标:")
    for i, point in enumerate(odd_points, start=1):
        print(f"奇数点 {i}: {point}")


C:\Users\SU_hole\AppData\Local\Temp\ipykernel_77952\3733585984.py:80: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


偶数点坐标:
偶数点 1: (125, 327)
偶数点 2: (228, 325)
偶数点 3: (329, 323)
偶数点 4: (431, 322)
偶数点 5: (532, 320)
偶数点 6: (634, 318)
偶数点 7: (735, 317)
偶数点 8: (837, 315)
偶数点 9: (938, 313)

奇数点坐标:
奇数点 1: (65, 296)
奇数点 2: (176, 294)
奇数点 3: (279, 292)
奇数点 4: (380, 290)
奇数点 5: (482, 289)
奇数点 6: (583, 287)
奇数点 7: (685, 285)
奇数点 8: (785, 283)
奇数点 9: (888, 281)
